In [2]:
import dash
from dash import dcc, html, Input, Output
import pandas as pd
import plotly.express as px

In [3]:
df = pd.read_csv('D:/DEPI/retail_store_inventory/retail_store_inventory.csv')
df['Date'] = pd.to_datetime(df['Date'])


In [4]:
# Initialize Dash App
app = dash.Dash(__name__)


app.layout = html.Div([
    html.H1('Retail Sales Dashboard', style={'textAlign': 'center'}),

    html.Div([
        html.Label('Select Category:'),
        dcc.Dropdown(
            id='category-dropdown',
            options=[{'label': cat, 'value': cat} for cat in df['Category'].unique()],
            multi=True,
            placeholder="Select Category"
        ),

        html.Label('Select Region:'),
        dcc.Dropdown(
            id='region-dropdown',
            options=[{'label': region, 'value': region} for region in df['Region'].unique()],
            multi=True,
            placeholder="Select Region"
        ), 
        
        html.Label('Select Date Range:'),
        dcc.DatePickerRange(
            id='date-range-picker',
            start_date=df['Date'].min(),
            end_date=df['Date'].max()
        ),
    ], style={'width': '25%', 'display': 'inline-block', 'verticalAlign': 'top', 'padding': '20px'}),

    html.Div([
        dcc.Graph(id='sales-trend-graph')
    ], style={'width': '70%', 'display': 'inline-block', 'padding': '20px'}),
])


In [5]:
# Callback for updating graph
@app.callback(
    Output('sales-trend-graph', 'figure'),
    Input('category-dropdown', 'value'),
    Input('region-dropdown', 'value'),
    Input('date-range-picker', 'start_date'),
    Input('date-range-picker', 'end_date')
)
def update_graph(selected_categories, selected_regions, start_date, end_date):
    # Filter dataset
    filtered_df = df.copy()

    if selected_categories:
        filtered_df = filtered_df[filtered_df['Category'].isin(selected_categories)]

    if selected_regions:
        filtered_df = filtered_df[filtered_df['Region'].isin(selected_regions)]

    filtered_df = filtered_df[(filtered_df['Date'] >= start_date) & (filtered_df['Date'] <= end_date)]

    # Group by Date
    sales_by_date = filtered_df.groupby('Date')['Units Sold'].sum().reset_index()

    # Plot
    fig = px.line(sales_by_date, x='Date', y='Units Sold', title='Daily Sales Trend')
    fig.update_traces(line_color='blue')

    return fig

# Run the app
if __name__ == '__main__':
    app.run(debug=True)